In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip  "gdrive/My Drive/result.zip"  "result/*"

In [0]:
!pip3 install -U gensim

In [3]:
import numpy as np
import math
import pandas as pd
import os
import codecs
import pickle
import gensim

from nltk.corpus import stopwords
import nltk
nltk.download("stopwords")
RUSSIANSTOPWORDS = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
class Lexicon():
    def __init__(self):
        self.dictionary = {}
        self.counter = 0
        
    
    def add(self, word):
        if (word in self.dictionary):
            return False
        
        self.dictionary[word] = self.counter
        self.counter += 1
        return True
    
    def get_num(self, word):
        if word in self.dictionary:
            return self.dictionary[word]
        else:
            return -1
    
lex = Lexicon()   
        

def get_forward_index(dirname):
    
    doc_id_txt = {}
    for idx,filename in enumerate(sorted(os.listdir(dirname))):
        if (idx % 1000 == 0):
            print(idx)
        number = int(filename.split('.')[0])
        with codecs.open(os.path.join(dirname, filename),'r','utf-8',errors='replace') as f:
            title = f.readline().rstrip()
            h = []
            for i in range(6):
                h.append(f.readline().rstrip())

            text = f.readline().rstrip()

        doc_id_txt[number] = (title, h, text) 

    return doc_id_txt



In [5]:
forward_index = get_forward_index('result/')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [0]:
def create_inverse_index(dirname):
    
    stats = dict()
    inverse_index = dict()
    filenames = os.listdir(dirname)
    for idx, filename in enumerate(filenames):
        if (idx % 1000 ==0):
            print(idx)
            
        docID = int(filename.split('.')[0])
        fullpath = os.path.join(dirname, filename)
        with codecs.open(fullpath, 'r', 'utf-8', errors='replace') as f:
            i = 0
            for line in f:
                for word in line.split():
                    i += 1
                    if word in inverse_index:
                        inverse_index[word].add(docID)
                    else:
                        inverse_index[word] = set([docID])
        stats[docID] = i
    return inverse_index, stats

def create_inverse_index2(forward_index):

    word_count = dict()
    inverse_index = dict()

    for idx, docID in enumerate(forward_index.keys()):
        if (idx % 1000 == 0):
            print(idx)
        
        title, h, text = forward_index[docID]
        all_text = title + ' ' +text
        for word in all_text.split():
            if (word in inverse_index):
                inverse_index[word].add(docID)
                word_count[word] += 1
            else:
                inverse_index[word] = set([docID])
                word_count[word] = 1

    return inverse_index, word_count

def create_inverse_index_bigrams(forward_index):
    bigram_count = dict()
    bigram_inverse_index = dict()

    for idx, docID in enumerate(forward_index.keys()):
        if (idx % 1000 == 0):
            print(idx)
        
        title, h, text = forward_index[docID]
        all_text = (title + ' ' +text).split()
        text_bigrams =  ['{} {}'.format(all_text[i], all_text[i+1]) 
                         for i in range(0, len(all_text)-1, 2)]

        for bigram in text_bigrams:
            if (bigram in bigram_inverse_index):
                bigram_inverse_index[bigram].add(docID)
                bigram_count[bigram] += 1
            else:
                bigram_inverse_index[bigram] = set([docID])
                bigram_count[bigram] = 1

    return bigram_inverse_index, bigram_count

In [8]:
inverse_index, word_count = create_inverse_index2(forward_index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000


In [0]:
len(inverse_index['казань'])

In [0]:
with open("inverse_index.pickle", 'wb') as f:
    pickle.dump(inverse_index,f)

with open('word_count.pickle','wb') as f:
    pickle.dump(word_count, f)

In [0]:
with open("forward_index.pickle", 'wb') as f:
    pickle.dump(forward_index, f)

In [0]:
with open("inverse_index.pickle", 'rb') as f:
    inverse_index = pickle.load(f)

In [0]:
with open('word_count.pickle','rb') as f:
    word_count = pickle.load(f)

In [0]:
with open('forward_index.pickle','rb') as f:
    forward_index = pickle.load(f)

In [0]:
#simple bm25
class BM25:

    def __init__(self, inverse_index, word_count, forward_index):
        self.inverse_index = inverse_index
        self.word_count = word_count
        #self.avgdl = sum([doc_stats[doc_id] for doc_id in doc_stats]) / len(doc_stats)
        self.forward_index = forward_index
        self.docs_total = len(self.forward_index)
        self.total_words = sum([word_count[key] for key in word_count.keys()])

    def get_important(self, words, words_idf, threshold = 0.9):


        if (len(words) <=  1):
            return words

        
        variants = [words]

        for variant in variants:
            total_idf = sum([words_idf[word] for word in variant])
            for idx, word in enumerate(variant):
                probable = variant[:idx] + variant[idx+1:]
                if ((total_idf - words_idf[word]) / total_idf > threshold and probable not in variants):
                    variants.append(probable)
                    
        return variants
         
    def computeScore(self, words, text, words_idf):
        def find_positions(words, text):

            wordset = set(words)
            words_positions = {word:[] for word in words}
            for idx, text_word in enumerate(text):
                if text_word in wordset:
                    words_positions[text_word].append(idx)

            return words_positions
        

        text = text.split()
        words_positions = find_positions(words, text)
        #print(words_positions)
        Nmiss = sum([1 for word in words_positions.keys() 
                        if not len(words_positions[word])])

        words = [word for word in words_positions.keys() if len(words_positions[word]) > 0]
        single_score = 0
        ''' 
        for idx, word in enumerate(words):
            word_idf = words_idf[word]
            word_freq = len(words_positions[word])
            word_score = 0
            for position in words_positions[word]:
                word_score += 1 / (1 + np.log(position + 1)) * word_idf

            word_score /=  (len(words_positions[word]) + 1e-6)
            single_score += word_score
        
        '''
        pair_score = 0
        for offset in range(0,2):
            
            for idx in range(len(words) - 1 - offset):
                prev_word = words[idx]
                next_word = words[idx + 1 + offset]
                if (prev_word == next_word) : continue
                pair_tf = 0
                i = 0
                j = 0
                pairs = 0
                while i < len(words_positions[prev_word]) and j < len(words_positions[next_word]):
                    prev_pos = words_positions[prev_word][i]
                    next_pos = words_positions[next_word][j]
                    #print("pos:", prev_pos, next_pos)
                    if abs(prev_pos - next_pos) < 3:
                        pair_tf +=  1 /abs(prev_pos - next_pos)
                        i += 1
                        j += 1

                    elif (prev_pos > next_pos):
                        j += 1
                    else:
                        i += 1

                pair_score +=  0.2 * (min(words_idf[prev_word], words_idf[next_word])) * pair_tf/ (1 + pair_tf)
        
        all_words_score = 0
        for word in words:
           all_words_score += 0.1 * words_idf[word] * (0.03 ** Nmiss)


        #print("words:", words)
        #print("single: {}  all: {}".format(single_score, all_words_score))

        score = pair_score + all_words_score

        return score 


    def __call__(self, query, q_id):


        splitted = query.split()
        words_idf = {}
        words = []

        for word in splitted:
            if word in inverse_index:
                words.append(word)
                words_idf[word] = np.log(self.total_words / len(inverse_index[word]))

        all_docs = set()
        threshold = 0.7

        if (len(words) > 12):
            wordidf = [(word, words_idf[word]) for word in words]
            wordidf = sorted(wordidf, key=lambda x:x[1], reverse=True)[:12]
            wordidf = [word[0] for word in wordidf]
            words = [word for word in words if word in wordidf]

        while (len(all_docs) < 10):
            
                print(threshold)
                #print(len(all_docs))
                probable_queries = self.get_important(words, words_idf, threshold)
                #print(probable_queries)
                for probable_query in probable_queries:

                    relevant = self.inverse_index[probable_query[0]]
                    for word in probable_query[1:]:
                        
                        relevant = relevant.intersection(self.inverse_index[word])
                    
                    all_docs |= relevant
                
                #print("ALL", len(all_docs))
                threshold -= 0.1
            
        #print(all_docs)
                all_docs = set(
                        [docID for docID in all_docs 
                        if (docID > 96*q_id - 300) and (docID < 96*q_id + 300)]
                        )
        docID_score = []

        title_corpus = []
        text_corpus_unigrams = []    
        text_corpus_bigrams = []
        docIDs = [docID for docID in all_docs]
        my_score = []
        
        for docID in docIDs:
            
            
            title, h, text = self.forward_index[docID]

            title_corpus.append(title.split())
            splitted_text = text.split()
            text_corpus_unigrams.append(splitted_text)
            text_corpus_bigrams.append([' '.join(splitted_text[i:i+2]) 
                                for i in range(0, len(splitted_text), 2)]
                                ) 
            my_score.append(self.computeScore(words, self.forward_index[docID][-1], words_idf))
        
        bm25_text_bigrams = BM25Okapi(text_corpus_bigrams)

        bm25_title = BM25Okapi(title_corpus)
        title_scores = bm25_title.get_scores(words)

        bm25_text_unigrams = BM25Okapi(text_corpus_unigrams)
        text_scores_unigrams = bm25_text_unigrams.get_scores(words)


        total_score = [tscore + txtscore
                        for tscore, txtscore, myscore in zip(title_scores, text_scores_unigrams, my_score)]
        docID_score = sorted([(docID, score) for docID, score in zip(docIDs, total_score)], key=lambda x:x[1], reverse=True)
        return [docID_score[i][0] for i in range(10)]

In [0]:
#doc_id_text.keys()[:10]

ranger = BM25(inverse_index, word_count, forward_index)
ranger("где можно постирать ковер", 110)

0.7


[10551, 10568, 10591, 10529, 10597, 10613, 10544, 10588, 10550, 10579]

In [0]:
#create 4 workers for ranking
import multiprocessing 

def worker(job_id):
    ranger = BM25(inverse_index, word_count, doc_id_text)
    with open('./gdrive/My Drive/queries.txt', 'r') as fr:
        with open('./{}result.txt'.format(job_id // 100) ,'w') as fw: 
            fw.write("QueryId,DocumentId\n")
            for idx, line in enumerate(fr):
                if not(idx >= job_id and job_id + 100 > idx) : continue
                

                query_id, query = int(line.split()[0]), ' '.join(line.split()[1:])
                try:
                    result = ranger(query, query_id)
                except:
                    print("exception occured in ", query_id)

                for docID in result:
                    fw.write("{},{}\n".format(query_id, docID))



processes = []
for i in range(0, 400, 100):
    p = multiprocessing.Process(target=worker, args=(i,))
    processes.append(p)
    p.start()
    
for process in processes:
    process.join()


In [0]:
def unite():
    with open("final.csv", 'w') as f:
        f.write("QueryId,DocumentId\n")
        for i in range(4):
            with open("{}result.txt".format(i), 'r') as rf:
                next(rf)
                for line in rf:
                    f.write(line)
unite()

# New Section

In [0]:
!wget http://vectors.nlpl.eu/repository/11/196.zip

--2019-12-15 10:32:25--  http://vectors.nlpl.eu/repository/11/196.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206986345 (197M) [application/zip]
Saving to: ‘196.zip’

196.zip             100%[===================>] 197.40M  9.86MB/s    in 27s     

2019-12-15 10:32:54 (7.26 MB/s) - ‘196.zip’ saved [206986345/206986345]



In [9]:
!pip3 install -q deeppavlov
!python3 -m deeppavlov install squad_bert
!python3 -m deeppavlov install ru_odqa_infer_wiki

     |████████████████████████████████| 737kB 5.1MB/s 
     |████████████████████████████████| 25.2MB 159kB/s 
     |████████████████████████████████| 317kB 45.5MB/s 
     |████████████████████████████████| 10.1MB 37.5MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 6.7MB 17.3MB/s 
     |████████████████████████████████| 2.1MB 29.2MB/s 
     |████████████████████████████████| 8.0MB 15.7MB/s 
     |████████████████████████████████| 163kB 47.7MB/s 
     |████████████████████████████████| 17.3MB 198kB/s 
     |████████████████████████████████| 2.8MB 39.3MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 2.3MB 49.4MB/s 
     |████████████████████████████████| 7.1MB 17.1MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
    

In [0]:
#BERT Model for Ranking

from shutil import copyfile

from deeppavlov import configs
from deeppavlov.core.common.file import read_json
from deeppavlov import  train_model, build_model

class PavlovRanker:

    def __init__(self, inverse_index, word_count, forward_index, BertModel):

        #------for computing idf-----
        self.inverse_index = inverse_index
        self.word_count = word_count
        #----------------------------------

        self.forward_index = forward_index


        self.docs_total = len(self.forward_index)
        self.total_words = sum([word_count[key] for key in word_count.keys()])

        self.model = BertModel 

    
    def computeScore(self, words, text, words_idf, avgdl):
        '''
        Classic bm25okapi scorer function with position bonus
        '''

        #-----addition function for searching word's positions------------------
        def find_positions(words, text):

            wordset = set(words)
            words_positions = {word:[] for word in words}
            for idx, text_word in enumerate(text):
                if text_word in wordset:
                    words_positions[text_word].append(idx)

            return words_positions
        
        #-----------------------------------------------------------------------
        k1 = 2.0
        b = 0.75
        #-----------------------------------------------------------------------
        text = text.split()
        words_positions = find_positions(words, text)
        Nmiss = sum([1 for word in words_positions.keys() 
                        if not len(words_positions[word])])

        words = [word for word in words_positions.keys() if len(words_positions[word]) > 0]
        single_score = 0
        
        #----------------------Single score computation-------------------------
        for idx, word in enumerate(words):
            word_idf = words_idf[word]
            word_freq = len(words_positions[word])
            tf = len(words_positions[word])

            word_score = word_idf * tf * (1 + k1) / (tf + k1*(1-b + b *len(text)/avgdl))

            position_score = 0

            for position in words_positions[word]:
                position_score +=  word_idf * (1 - position_score/len(text))

            position_score /= len(words_positions[word])
            word_score += position_score
            single_score += word_score
        
        #------------------------Pair score computation-------------------------
        pair_score = 0
        for offset in range(0,2):
            
            for idx in range(len(words) - 1 - offset):
                prev_word = words[idx]
                next_word = words[idx + 1 + offset]
                if (prev_word == next_word) : continue
                pair_tf = 0
                i = 0
                j = 0
                pairs = 0
                while i < len(words_positions[prev_word]) and j < len(words_positions[next_word]):
                    prev_pos = words_positions[prev_word][i]
                    next_pos = words_positions[next_word][j]
                    if abs(prev_pos - next_pos) < 3:
                        pair_tf +=  1 /abs(prev_pos - next_pos)
                        i += 1
                        j += 1

                    elif (prev_pos > next_pos):
                        j += 1
                    else:
                        i += 1

                pair_score +=  0.2 * (min(words_idf[prev_word], words_idf[next_word])) * pair_tf/ (1 + pair_tf)


        #-------------Additional Bonus for all word's presence------------------
        all_words_score = 0
        for word in words:
           all_words_score += 0.1 * words_idf[word] * (0.03 ** Nmiss)
        #-----------------------------------------------------------------------


        score = single_score + pair_score + all_words_score

        return score 

    
    def __call__(self, query, q_id):


        splitted = query.split()
        words_idf = {}
        words = []

        for word in splitted:
            if word in inverse_index:
                words.append(word)
                words_idf[word] = np.log(self.total_words / len(inverse_index[word]))

            
    
        all_docs = [i for i in range(96*q_id - 100, 96*q_id + 50, 1)]

        avg_dl_text = sum([len(forward_index[doc_id][-1].split()) for doc_id in all_docs]) / len(all_docs)
        avg_dl_title = sum([len(forward_index[doc_id][0].split()) for doc_id in all_docs]) / len(all_docs)

        docID_score = []

        for idx, docID in enumerate(all_docs):
            
            title = self.forward_index[docID][0]
            text = self.forward_index[docID][-1]

            bm25_title_score = self.computeScore(words, title, words_idf, avg_dl_title) 
            bm25_text_score = self.computeScore(words,text, words_idf, avg_dl_text)
            answer = self.model([text], query)
            answer_score = answer[-1][0]
            print(idx, bm25_title_score, bm25_text_score, answer_score)
            total_score = 5 * bm25_title_score + bm25_title_score + 3 * answer_score
            docID_score.append((docID, total_score))


        top10 = sorted(docID_score, key=lambda x:x[1], reverse=True)[:10]
        docIDs = [word_score[0] for word_score in top10]
        return docIDs


In [11]:
model_config = read_json(configs.squad.squad_ru_bert_infer)
model = build_model(model_config, download=True)

2020-01-11 16:33:57.38 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/squad_ru_bert.tar.gz to /root/.deeppavlov/squad_ru_bert.tar.gz
100%|██████████| 660M/660M [02:59<00:00, 3.67MB/s]
2020-01-11 16:36:56.933 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/squad_ru_bert.tar.gz archive into /root/.deeppavlov/models
2020-01-11 16:37:04.510 INFO in 'deeppavlov.core.data.utils'['utils'] at line 80: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [03:05<00:00, 3.58MB/s]
2020-01-11 16:40:09.623 INFO in 'deeppavlov.core.data.utils'['utils'] at line 237: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   

Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-01-11 16:40:50.17 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/squad_ru_bert/model_multi]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/squad_ru_bert/model_multi


In [0]:
pr = PavlovRanker(inverse_index, word_count, forward_index, model)

In [13]:
pr('как ставить значок на клавиатура', 138)

0 0 28.072119732881433 0.22750252485275269
1 0 90.18403880226901 9.907266616821289
2 0 56.435155232343476 7.249207496643066
3 45.12429835167505 90.89101751730858 0.6031476855278015
4 0 61.987257762712545 6.002791404724121
5 18.96857141452895 67.14842943243703 0.061496444046497345
6 14.902390629974294 65.43424557158836 11.904730796813965
7 21.16592543426699 68.41056966116327 2.044391632080078
8 0 67.02291567955429 3.9562740325927734
9 0 66.3179967743185 0.02281741425395012
10 0 85.14364964010348 6.519864082336426
11 0 92.94245130384961 0.710308849811554
12 20.32818640773533 69.87849509820217 0.19885209202766418
13 0 92.2080903771893 0.06445620208978653
14 23.301980838508364 66.86456901777618 111.61415100097656
15 20.32818640773533 68.54385517947541 0.13884620368480682
16 0 70.1466238599872 4.306094646453857
17 0 28.947309634941547 0.052763402462005615
18 0 64.80600095038541 2.8415889739990234
19 0 66.88485782475665 0.9111777544021606
20 0 65.50301603038223 19.368316650390625
21 17.06866

[13282, 13292, 13285, 13293, 13297, 13294, 13269, 13283, 13289, 13286]

In [0]:
ranger = PavlovRanker(inverse_index, word_count, forward_index, model)
with open('./gdrive/My Drive/queries.txt', 'r') as fr:
    with open('./result.csv', 'w') as fw: 
        fw.write("QueryId,DocumentId\n")
            for idx, line in enumerate(fr):
                

                query_id, query = int(line.split()[0]), ' '.join(line.split()[1:])
                try:
                    result = ranger(query, query_id)
                except:
                    print("exception occured in ", query_id)

                for docID in result:
                    fw.write("{},{}\n".format(query_id, docID))



def get_dict(filename)
    finded = {}
    with open(filename, 'r') as f:
        f.readline()
        for line in f:
            q_id, doc_id = line.split()
            q_id = int(q_id)
            doc_id = int(doc_id.rstrip())
            if (q_id not in finded):
                finded[q_id] = [doc_id]
            else:
                finded[q_id].append(doc_id)

        return finded

finded = get_dict('./gdrive/My Drive/finded.csv')
ranked = get_dict('/gdrive/My Drive/result.csv')

with open('try10.csv', 'w') as f:
    f.write('QueryId,DocumentId')
    for key in sorted(ranked.keys()):
        i = 0
        writed = []
        if (key in finded):
            
            for elem in finded[key]:
                i += 1
                f.write('{},{}\n'.foramt(key, elem))
                writed.append(elem)

        while (i < 10):
            for elem in ranked[key]:
                if (elem not in writed):
                    i += 1
                    f.write('{},{}\n'.foramt(key, elem))
                if (i == 10):
                    break


In [0]:
!wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lemmatize/ft_native_300_ru_wiki_lenta_lemmatize.vec

--2019-12-19 18:50:46--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lemmatize/ft_native_300_ru_wiki_lenta_lemmatize.vec
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2581071412 (2.4G) [application/octet-stream]
Saving to: ‘ft_native_300_ru_wiki_lenta_lemmatize.vec’

mmatize.vec          76%[==============>     ]   1.85G  5.27MB/s    eta 73s    ^C


In [0]:
#try use word2vec, but it works bad
class BMWord2vec:

    def __init__(self, inverse_index, words_idf ,forward_index, word2vec):
        self.inverse_index = inverse_index
        self.word_count = word_count
        self.forward_index = forward_index
        self.words_idf = words_idf
        self.word2vec = word2vec

    def get_important(self, words, words_idf, threshold = 0.9):


        if (len(words) <=  1):
            return words

        
        variants = [words]

        for variant in variants:
            total_idf = sum([self.words_idf[word] for word in variant])
            for idx, word in enumerate(variant):
                probable = variant[:idx] + variant[idx+1:]
                if ((total_idf - self.words_idf[word]) / total_idf > threshold and probable not in variants):
                    variants.append(probable)
                    
        return variants
         
    def computeScore(self, query_words, text):
        def computeCos(vec1, vec2):
            vec1_norm = np.sqrt(np.sum(vec1 ** 2))
            vec2_norm = np.sqrt(np.sum(vec2 ** 2))

            cos = np.sum(vec1 * vec2) / (vec1_norm * vec2_norm)

            return cos
        
        score = 0
        text = text.split()
        for word in query_words:
            vector_exist = False
            if word in self.word2vec:
                word_vec = np.asarray(self.word2vec[word])
                vector_exist = True

            for text_word in text:
                text_exist = False
                if (text_word in self.word2vec):
                    text_vec = np.asarray(self.word2vec[text_word])
                    text_exist = True

                if (text_exist and vector_exist and word in words_idf and text_word in words_idf):
                    pair_score = computeCos(text_vec, word_vec)
                    #print(pair_score, word, text_word)
                    score += pair_score * min(self.words_idf[text_word], self.words_idf[word])

                elif (text_word == word and word in words_idf):
                    score += ( words_idf[word])
        
        score = score / (len(query_words) * len(text))

        return score
   


    def __call__(self, query, q_id):


        splitted = query.split()
        words_idf = {}
        words = []

        for word in splitted:
            if word in self.words_idf:
                words.append(word)


        all_docs = set()
        threshold = 0.7

        if (len(words) > 12):
            wordidf = [(word, words_idf[word]) for word in words]
            wordidf = sorted(wordidf, key=lambda x:x[1], reverse=True)[:12]
            wordidf = [word[0] for word in wordidf]
            words = [word for word in words if word in wordidf]

        while (len(all_docs) < 10):
            
                print(threshold)
                probable_queries = self.get_important(words, words_idf, threshold)
 
                for probable_query in probable_queries:

                    relevant = self.inverse_index[probable_query[0]]
                    for word in probable_query[1:]:
                        
                        relevant = relevant.intersection(self.inverse_index[word])
                    
                    all_docs |= relevant
                
                threshold -= 0.1
            
                all_docs = set(
                        [docID for docID in all_docs 
                        if (docID > 96*(q_id-1) - 50) and (docID < (96*q_id-1) + 150)]
                        )
                
        docID_score = []
        for docID in all_docs:
            
            all_text = self.forward_index[docID][0] + ' ' + self.forward_index[docID][-1]
            score = self.computeScore(words, all_text)
            docID_score.append((docID, score))
            #print(docID, score)

        docID_score = sorted(docID_score, key=lambda x:x[1], reverse=True)[:10]
        docIDs = [elem[0] for elem in docID_score]

        return docIDs


def compute_words_idf(word_count):
    total_words = sum([word_count[word] for word in word_count.keys()])
    words_idf = {}
    for word in word_count.keys():
        words_idf[word] = np.log(total_words / word_count[word])

    return words_idf

def loadWord2vec(path):
    with open(path, 'r') as f:
        f.readline()
        word2vec = {}
        for line in f:
            word, vector = line.split()[0], line.split()[1:]
            vector = [float(val) for val in vector]
            word2vec[word] = vector
        return word2vec

In [0]:
word2vecs = loadWord2vec('ft_native_300_ru_wiki_lenta_lemmatize.vec')

In [0]:
words_idf = compute_words_idf(word_count)

In [0]:
ranker = BMWord2vec(inverse_index, words_idf, forward_index, word2vecs)
